In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *

In [ ]:
#make working folders to more easily organize the training and validation sets
benign_path = Path('/kaggle/working/data/benign')
benign_path.mkdir(parents=True, exist_ok=True)
malicious_path = Path('/kaggle/working/data/malicious')
malicious_path.mkdir(parents=True, exist_ok=True)

#copy only the 1200 DPI lanczos interpolation images from the dataset
!cp -r /kaggle/input/malware-as-images/data/benign/lanczos_1200/* {benign_path}/
!cp -r /kaggle/input/malware-as-images/data/malicious/lanczos_1200/* {malicious_path}/

In [ ]:
path = Path('/kaggle/working/data/')

In [ ]:
#no reason to spin/distort to mimic different camera angles and such
dls = ImageDataLoaders.from_folder(path, train=".", valid_pct=0.2, 
                                   item_tfms=Resize(256), 
                                   bs=16, seed=42)
# dls.show_batch(nrows=3, ncols=3)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find(stop_div=False, num_it=200)

In [ ]:
learn.fit_one_cycle(4, lr_max=slice(1e-5,1e-1))

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.export()